# Photometrically calibrating astronomy images using catalogs of known stars.

In this notebook we explain in detail how the the package ```photcal``` and how you can use it to calibrate your own data. First we derive a general formalization for calibration and then discuss how these are implemented in ```photcal```. 

## Catalogs

One of the most common methods of photometrically calibrating astronomy images is too identify stars within your image and then find them in existsing photometric catalogs like GAIA, PANSTARRS, or SDSS. The choose of which stars in your image you choose and which catalog you use is entirely up to you. These choices should be made intelligently. For example, you don't want to choose stars in your image that have been saturated. On the other hand you don't want to choose stars that are too faint such that they aren't well observed at the depth of your chosen catalog.  

Once the stars have been selected---using either a software like sextractor(https://www.astromatic.net/software/sextractor/) or DAOstarfinder (https://photutils.readthedocs.io/en/stable/detection.html) then you can compare their instrumental magnitudes with the values in the catalog and build a relationship. 

```photcal``` is useful when this process has been completed and the user has a source catalog of their own image as well corresponding catalog magnitudes.

## Building a relationship

Naively it makes sense to think that if we have the instrumental magnitudes and their corresponding magnitudes in a catalog then we can derive a simple, linear relationship between the two. In this case they would be offset by a constant value such that 

\begin{equation}
m_{\rm inst} = c_{0} + m_{\rm catalog},
\end{equation}

where $m$ would be the magnitude in whichever band you've chosen and $c_0$ is a constant. However, this doesn't take into account that the filters used for the catalog are---most likely---different from the filters used in your own image; the shape of the DECam $i$ filter is slightly different to the $i$ filter used by PANSTARRS.

We can account for this difference by including several color terms and improve the above equation.

\begin{equation}
m_{\rm inst} = c_0 + m_{\rm catalog} + c_1\left(\Delta T_{1}\right) + c_2\left(\Delta T_{2}\right) + \cdots.
\end{equation}

Here the $\Delta T$ values are the color, i.e., the difference between different filters of the source within the catalog. For example $i-z$ or $z-r$. Note also that each color term is associated with a new constants. We can formulize the above more concretely by taking into account the fact that this relationship should hold for any galaxy $i$ for any number of color terms:

\begin{equation}
m_{\rm inst, i} = c_0 + m_{\rm catalog, i} + \sum \limits_{k=1}^{N_{t}} c_k\left(\Delta T_{k, i}\right),
\end{equation}

where $N_{t}$ is the number of color terms that have been chosen. It's important to note that the $c_{k}$ values are not dependednt on $i$, that is to say that they are truly constant for all the galaxies in the sample. Performing the calibration relies on determining the value of the constants. To do this we opt to minimize a $\chi ^2$ distribution.

## Minimizing the chi-squared value

$\chi ^{2}$ is defined as:

\begin{equation}
\chi ^2 \equiv \sum  \limits_{i = 1}^{N} \frac{\left( y_i - y_{i,E} \right)^2}{\sigma ^2_i}
\end{equation}

Where $y_i$ is the observed value of galaxy $i$ (in this case the instrumental magnitudes) and $y_{i, E}$ is the expected value of galaxy $i$ (in this case the right hand side of the equation we derived). (see https://arxiv.org/pdf/astro-ph/0310577.pdf).

substituting appropriately we have:

\begin{equation}
\chi ^2 \equiv \sum  \limits_{i = 1}^{N} \frac{\left[ m_{\rm inst, i} - \left( c_0 + m_{\rm catalog, i} + \sum \limits_{k=1}^{N_{t}} c_k\left(\Delta T_{k, i}\right) \right) \right]^2}{\sigma ^2_i}
\end{equation}

Minimizing $\chi^2$ means that we need to find where the partial derivitives are equal to zero. We can do this for $c_0$ such that:

\begin{align}
\frac{\partial \chi^2}{\partial c_0} &= 0 \\
\implies \frac{\partial}{\partial c_0} \sum  \limits_{i = 1}^{N} \frac{\left[ m_{\rm inst, i} - \left( c_0 + m_{\rm catalog, i} + \sum \limits_{k=1}^{N_{t}} c_k\left(\Delta T_{k, i}\right) \right) \right]^2}{\sigma ^2_i}  &= 0 \\
\implies -2 \sum \limits_{i = 1}^{N} \frac{\left[ m_{\rm inst, i} - \left( c_0 + m_{\rm catalog, i} + \sum \limits_{k=1}^{N_{t}} c_k\left(\Delta T_{k, i}\right) \right)   \right]}{\sigma ^2_i} &=0
\end{align}

If we define $\Delta T_0 \equiv 1$, then the above equation becomes:

\begin{align}
-2 \sum \limits_{i = 1}^{N} \frac{\left[ m_{\rm inst, i} -  m_{\rm catalog, i} - \sum \limits_{j=0}^{N_{t}} c_j\left(\Delta T_{j, i}\right)   \right]}{\sigma ^2_i} &=0 \\
\implies \sum \limits_{i = 1}^{N} \left( \frac{m_{\rm inst, i} -  m_{\rm catalog, i}}{\sigma_i^2} \right) &= \sum \limits_{i = 1}^{N} \frac{\sum \limits_{j=0}^{N_{t}} c_j\left(\Delta T_{j, i}\right)}{\sigma ^2_i}
\end{align}

In a similar fashion, $\frac{\partial \chi^2}{\partial c_1} = 0$ leads to
\begin{equation}
\implies \sum \limits_{i = 1}^{N} \left[ \frac{\left(m_{\rm inst, i} -  m_{\rm catalog, i}\right) \Delta T_1}{\sigma_i^2} \right] = \sum \limits_{i = 1}^{N} \frac{\sum \limits_{j=0}^{N_{t}} c_j\left(\Delta T_{j, i}\right)  \Delta T_1} {\sigma ^2_i}
\end{equation}

Unsurprisingly $\frac{\partial \chi^2}{\partial c_2} = 0$ results in
\begin{equation}
\implies \sum \limits_{i = 1}^{N} \left[ \frac{\left(m_{\rm inst, i} -  m_{\rm catalog, i}\right) \Delta T_2}{\sigma_i^2} \right] = \sum \limits_{i = 1}^{N} \frac{\sum \limits_{j=0}^{N_{t}} c_j\left(\Delta T_{j, i}\right)  \Delta T_2} {\sigma ^2_i}
\end{equation}

There for for any given color term $j$ (remembering of course that $\Delta T_0 \equiv 0$), the minimization of $\frac{\partial \chi^2}{\partial c_j} = 0$ gives us the equation:

\begin{equation}
\implies \sum \limits_{i = 1}^{N} \left[ \frac{\left(m_{\rm inst, i} -  m_{\rm catalog, i}\right) \Delta T_j}{\sigma_i^2} \right] = \sum \limits_{i = 1}^{N} \frac{\sum \limits_{j=0}^{N_{t}} c_j\left(\Delta T_{j, i}\right)  \Delta T_j} {\sigma ^2_i}
\end{equation}


Therefore for any given $N_t$ number of color terms we have a corresponding set of $N_t$ equations with $N_t$ number of unknown values---$c_j$. This system of equations can then be solved.

## Solving the system of equations

The previous discussion can be described using linear algerbra as:

\begin{equation}
\vec{x} = A\vec{c}
\end{equation}

where $\vec{x}$ is a vector representation the left hand terms of the general equation we wrote, $\vec{c}$ represents the constants, and $A$ is a $N_t \times N_t$ matrix, representing the right hand side of the general equation. 

each component of $\vec{x}$ is:
\begin{equation}
x_j = \sum \limits_{i=1}^{N} \frac{\Delta m \Delta T_j}{\sigma^2}
\end{equation}

The elements of matrix $A$ can be determined as:

\begin{equation}
A_{ab} = \sum \limits_{i=1}^{N} \frac{\Delta T_a \Delta T_b}{\sigma^2}
\end{equation}

Then the constants, $\vec{c}$ can then be determined by $\vec{c} = A^{-1}\vec{x}$